In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import glob, os
from IPython.display import Image
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR, SVR
from sklearn import metrics
from sklearn.linear_model import LinearRegression,Ridge
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn import tree
import pydotplus
from sklearn.externals.six import StringIO
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression
from sklearn.feature_selection import f_regression
from sklearn.linear_model import ElasticNet
import forward_selection as fs
from scipy.stats import chi2_contingency, pearsonr, spearmanr

In [2]:
dft11 = pd.read_csv("la_base.csv", encoding="latin1")

In [3]:
dft11["taux_bgauche"] = dft11['taux_xgauche']+ dft11['taux_gauche'] + dft11['taux_vert']
dft11["taux_bdroite"] = dft11['taux_droite']+ dft11['taux_centre']

In [4]:
#Taux_supp_moyenne sortant
#Popularité sortant 

dft11["ecart_pop"] = dft11["pop_president"] - dft11["pop_premier_ministre"] 
dft11["pop_exec"] = np.nan 
dft11.ix[dft11["Ann?e"]==1981,"pop_exec"] = dft11.ix[dft11["Ann?e"]==1981,"pop_president"].iloc[0]
dft11.ix[dft11["Ann?e"]==1988,"pop_exec"] = dft11.ix[dft11["Ann?e"]==1988,"pop_president"].iloc[0]
dft11.ix[dft11["Ann?e"]==2012,"pop_exec"] = dft11.ix[dft11["Ann?e"]==2012,"pop_president"].iloc[0]
dft11.ix[dft11["Ann?e"]==1995,"pop_exec"] = dft11.ix[dft11["Ann?e"]==1995,"pop_premier_ministre"].iloc[0]
dft11.ix[dft11["Ann?e"]==2002,"pop_exec"] = dft11.ix[dft11["Ann?e"]==2002,"pop_premier_ministre"].iloc[0]
dft11.ix[dft11["Ann?e"]==2007,"pop_exec"] = dft11.ix[dft11["Ann?e"]==2007,"pop_premier_ministre"].iloc[0]

dft11.ix[dft11["Ann?e"]==1981,"pop_xdroite"] = dft11.ix[dft11["Ann?e"]==1988,"pop_xdroite"].iloc[0]
dft11.ix[dft11["Ann?e"]==1981,"pop_verts"] = dft11.ix[dft11["Ann?e"]==1995,"pop_verts"].iloc[0]
dft11.ix[dft11["Ann?e"]==1988,"pop_verts"] = dft11.ix[dft11["Ann?e"]==1995,"pop_verts"].iloc[0]

dft11.ix[dft11["Ann?e"]==1981,"nombre de logement vacant"] = 1891.

In [5]:
dft11["taux_sortie_sans_bloc"] = np.nan 
dft11.ix[dft11["Ann?e"]==1981,"taux_sortie_sans_bloc"] = dft11.ix[dft11["Ann?e"]==1981,"taux_centre"]
dft11.ix[dft11["Ann?e"]==1988,"taux_sortie_sans_bloc"] = dft11.ix[dft11["Ann?e"]==1988,"taux_gauche"]
dft11.ix[dft11["Ann?e"]==1995,"taux_sortie_sans_bloc"] = dft11.ix[dft11["Ann?e"]==1995,"taux_gauche"]
dft11.ix[dft11["Ann?e"]==2002,"taux_sortie_sans_bloc"] = dft11.ix[dft11["Ann?e"]==2002,"taux_droite"]
dft11.ix[dft11["Ann?e"]==2007,"taux_sortie_sans_bloc"] = dft11.ix[dft11["Ann?e"]==2007,"taux_droite"]
dft11.ix[dft11["Ann?e"]==2012,"taux_sortie_sans_bloc"] = dft11.ix[dft11["Ann?e"]==2012,"taux_droite"]

In [6]:
dft11["taux_sortie_avec_bloc"] = np.nan 
dft11.ix[dft11["Ann?e"]==1981,"taux_sortie_avec_bloc"] = dft11.ix[dft11["Ann?e"]==1981,"taux_centre"] +\
dft11.ix[dft11["Ann?e"]==1981,"taux_droite"]
dft11.ix[dft11["Ann?e"]==1988,"taux_sortie_avec_bloc"] = dft11.ix[dft11["Ann?e"]==1988,"taux_gauche"]+\
dft11.ix[dft11["Ann?e"]==1988,"taux_xgauche"] + dft11.ix[dft11["Ann?e"]==1988,"taux_vert"]
        
dft11.ix[dft11["Ann?e"]==1995,"taux_sortie_avec_bloc"] = dft11.ix[dft11["Ann?e"]==1995,"taux_gauche"]+\
dft11.ix[dft11["Ann?e"]==1995,"taux_xgauche"] + dft11.ix[dft11["Ann?e"]==1995,"taux_vert"]

dft11.ix[dft11["Ann?e"]==2002,"taux_sortie_avec_bloc"] = dft11.ix[dft11["Ann?e"]==2002,"taux_droite"]+\
dft11.ix[dft11["Ann?e"]==2002,"taux_centre"]
dft11.ix[dft11["Ann?e"]==2007,"taux_sortie_avec_bloc"] = dft11.ix[dft11["Ann?e"]==2007,"taux_droite"]+\
dft11.ix[dft11["Ann?e"]==2007,"taux_centre"]
dft11.ix[dft11["Ann?e"]==2012,"taux_sortie_avec_bloc"] = dft11.ix[dft11["Ann?e"]==2012,"taux_droite"]+\
dft11.ix[dft11["Ann?e"]==2012,"taux_centre"]

In [7]:
def create_train(target, dft11, option_fn=False):
    dft11["CORS88"] = dft11.apply(lambda row : 1 if row["Ann?e"]==1988 and 
                              row["code"] in ["2A","2B"] else 0, axis=1)

    dft11["CORS02"] = dft11.apply(lambda row : 1 if row["Ann?e"]==2002 and 
                              row["code"] in ["2A","2B"] else 0, axis=1)
    
    dft12 = dft11.drop(["code", 'taux_Blancs et nuls', 'taux_droite', 'taux_xgauche',\
                    'taux_vert','taux_centre', 'Q1_rate', 'Q2_rate', 'Q3_rate', 'Q4_rate', "Total", \
                   'depart_OM', "Date","taux_Abstention_sup_moyenne",'taux_Blancs et nuls_sup_moyenne',
                        'depart_CORSE'], axis=1)
    
    dft12["Date"] = dft12["Ann?e"] - 1981
    


    
    list_dell = ["Ann?e","d?partement","taux_bgauche",'taux_Abstention',"taux_sortie_sans_bloc",
                "taux_sortie_avec_bloc","taux_gauche", 'taux_xdroite',"taux_bdroite"]

    
    
    
    if option_fn==True:
        dft12 = dft12[dft12["Ann?e"]!=1981]
        

    
    df_train_x = dft12.drop(list_dell, axis=1)
    df_train_y = dft12[target]
    
    return df_train_x, df_train_y

In [8]:
def create_train_and_test(target, dft11, option_fn=False):

    dft11["CORS88"] = dft11.apply(lambda row : 1 if row["Ann?e"]==1988 and 
                              row["code"] in ["2A","2B"] else 0, axis=1)

    dft11["CORS02"] = dft11.apply(lambda row : 1 if row["Ann?e"]==2002 and 
                              row["code"] in ["2A","2B"] else 0, axis=1)
    
    dft12 = dft11.drop(["code", 'taux_Blancs et nuls', 'taux_droite', 'taux_xgauche',\
                    'taux_vert','taux_centre', 'Q1_rate', 'Q2_rate', 'Q3_rate', 'Q4_rate', "Total", \
                   'depart_OM', "Date","taux_Abstention_sup_moyenne",'taux_Blancs et nuls_sup_moyenne',
                        'depart_CORSE'], axis=1)
    
    dft12["Date"] = dft12["Ann?e"] - 1981
    


    
    list_dell = ["Ann?e","d?partement","taux_bgauche",'taux_Abstention',"taux_sortie_sans_bloc",
                "taux_sortie_avec_bloc","taux_gauche", 'taux_xdroite',"taux_bdroite"]

    
    df_train = dft12[dft12["Ann?e"]!=2012]
    
    if option_fn==True:
        df_train = df_train[df_train["Ann?e"]!=1981]
        
    df_test = dft12[dft12["Ann?e"]==2012]
    
    df_train_x = df_train.drop(list_dell, axis=1)
    df_train_y = df_train[target]
    df_test_x = df_test.drop(list_dell, axis=1)
    df_test_y = df_test[target]
    
    return df_train_x, df_train_y, df_test_x, df_test_y

In [ ]:
features_keep = ["Nombre total de mariages domicili?s",
"taux_droite_sup_moyenne",
"taux_gauche_sup_moyenne",
"taux_xdroite_sup_moyenne",
"pop_xdroite",
"Densit?",
"subventions",
"pop_exec"
]

df_train_x, df_train_y, df_test_x, df_test_y = create_train_and_test("taux_xdroite", dft11, False)
df_train_x_select_features = df_train_x.ix[:, features_keep]
df_test_x_select_features = df_test_x.ix[:, features_keep]

lr = LinearRegression(fit_intercept=True, normalize=True)
lr.fit(df_train_x_select_features, df_train_y)
print(df_test_x_select_features.columns)
print("Résultat sans 2012 (avec test)")
print("R2 adjust train %s" % r2_adjusted(df_train_x_select_features, df_train_y, 8,lr))
print("R2 train %s" % lr.score(df_train_x_select_features, df_train_y))
print("MAE train %s" % metrics.mean_absolute_error(df_train_y, lr.predict(df_train_x_select_features)))
print("MAE test %s" % metrics.mean_absolute_error(df_test_y, lr.predict(df_test_x_select_features)))
print("\n")
print("Résultat avec fit entire data")
df_train_x, df_train_y = create_train("taux_xdroite", dft11, False)
df_train_x_select_features = df_train_x.ix[:, features_keep]
lr = LinearRegression(fit_intercept=True, normalize=True)
lr.fit(df_train_x_select_features, df_train_y)
print("R2 train %s" % lr.score(df_train_x_select_features, df_train_y))
print("R2 adjust train %s" % r2_adjusted(df_train_x_select_features, df_train_y, 8,lr))
print("MAE train %s" % metrics.mean_absolute_error(df_train_y, lr.predict(df_train_x_select_features)))